In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


In [12]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [15]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [19]:
layer_1 = nn.Linear(in_features=28*28, out_features=20)
hidden_1 = layer_1(flat_image)
print(hidden_1.size())

torch.Size([3, 20])


In [21]:
print(f"Before ReLU: {hidden_1}\n\n")
hidden_1 = nn.ReLU()(hidden_1)
print(f"After ReLU: {hidden_1}")

Before ReLU: tensor([[0.0238, 0.3280, 0.0000, 0.1012, 0.0000, 0.0000, 0.1838, 0.7940, 0.0000,
         0.1648, 0.0000, 0.0000, 0.3093, 0.5431, 0.0000, 0.0492, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.5072, 0.0000, 0.0000, 0.0000, 0.1689, 0.0000, 0.7466, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2381, 0.0000, 0.1695, 0.0000, 0.0000,
         0.0000, 0.2328],
        [0.0292, 0.6099, 0.0000, 0.0000, 0.0000, 0.0798, 0.2122, 0.7891, 0.0000,
         0.0000, 0.0000, 0.0000, 0.3104, 0.1041, 0.2886, 0.2566, 0.0000, 0.0000,
         0.0000, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0238, 0.3280, 0.0000, 0.1012, 0.0000, 0.0000, 0.1838, 0.7940, 0.0000,
         0.1648, 0.0000, 0.0000, 0.3093, 0.5431, 0.0000, 0.0492, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.5072, 0.0000, 0.0000, 0.0000, 0.1689, 0.0000, 0.7466, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2381, 0.0000, 0.1695, 0.0000, 0.0000,
         0.0000, 0.2328],
       

In [25]:
seq_modules = nn.Sequential(
    flatten,
    layer_1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [27]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [28]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0349, -0.0255,  0.0213,  ..., -0.0050,  0.0085,  0.0078],
        [-0.0245, -0.0002,  0.0314,  ...,  0.0087,  0.0293, -0.0184]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0268, -0.0348], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0402,  0.0180, -0.0014,  ...,  0.0341, -0.0405,  0.0250],
        [-0.0321,  0.0116,  0.0112,  ..., -0.0434,  0.0213,  0.0143]],
       device='cuda:0', grad_fn=<Sl